<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, deploy, and score a PMML model to predict species of iris</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to store a sample Predictive Model Markup Language (PMML) model and score test data. 
You will use the Iris data set to predict the species of an iris flower. This data set contains measurements of the iris perianth flower. 

Some familiarity with Python is helpful. This notebook uses `watson-machine-learning-client-V4` and is compatible with CP4D 3.0 and Python 3.6.


## Learning goals

You will learn how to:

-  Save a PMML model.
-  Deploy a PMML model.
-  Score the deployed model.


## Table of Contents

1.	[Set up the environment](#setup)<br>
2.	[Save the PMML model](#save)<br>
3.  [Create an online deployment (with deployment space only)](#deploy)<br>
4.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

To get started on CP4D 3.0, find documentation on installation and set up <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener no referrer">here</a>.

Import the `watson-machine-learning-client` module.
<div class="alert alert-block alert-info">
For more information about the <b>Watson Machine Learning Python client (V4)</b>, please refer to the <a href="https://wml-api-pyclient-dev-v4.mybluemix.net/" target="_blank" rel="noopener no referrer">Python client documentation</a>. If you're using the notebook within a project on your CP4D cluster, you do not need to install this package as it comes pre-installed with the notebooks. The installation code below is for demonstration but is non-executable at this stage.
</div>

In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**Authenticate the Python client on CP4D.**

<div class="alert alert-block alert-info">To find your authentication information (your credentials) follow the steps provided here in the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-authentication-local.html" target="_blank" rel="noopener no referrer">Documentation.</a></div>

In [3]:
# Enter your credentials here.

from project_lib.utils import environment
url = environment.get_common_api_url()

import sys,os,os.path
token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
     "instance_id": "openshift",
     "token": token,
     "url": url,
     "version": "3.0.0"
}

In [4]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [5]:
client.version

'1.0.267'

<div class="alert alert-block alert-info">
You have a choice to either save the model in the <b>project</b> or the <b>deployment space</b>:<br><br>
    <li> If you're saving the model in your project, you have to set the default project using the python client.</li><br>
    <li>If you're saving the model in the deployment space, you have to obtain the space UID of the deployment space you've created. Then you'd use this to set the default space using the python client. From there you'll be able to deploy and score the model in your deployment space.</li></div>

### To set the default project, use the following code.

In [6]:
from project_lib import Project
project = Project.access()
project_id = project.get_metadata()["metadata"]["guid"]

client.set.default_project(project_id)

'SUCCESS'

### To set the default space, follow these steps.

<div class="alert alert-block alert-info">
You can create your own <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/ml-spaces_local.html" target="_blank" rel="noopener no referrer">deployment space</a> by selecting <b>Analytics deployments</b> under <b>Analyze</b> from the Navigation Menu on the top left of this page.</div>

Alternatively, you can create a deployment and obtain its UID using the code in the following cell. The cell is not executable cell at this stage, but you can enter the name of your space in the metadata and use it if needed.

In [6]:
# Obtain the UId of your space
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['guid'])

**Action:** Enter the name of your deployment space in the code below: `space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')`.

In [7]:
# Enter the name of your deployment space here:
space_uid = guid_from_space_name(client, 'YOUR DEPLOYMENT SPACE')
print("Space UID = " + space_uid)

Space UID = d748238d-fd86-4db5-aa9d-3ebccad83f95


You can set the default space using the cell below.

In [8]:
client.set.default_space(space_uid)

'SUCCESS'

<a id="save"></a>
## 2. Save the PMML model

Use `wget` to download the sample PMML model, `iris_chaid.xml` from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.  

<div class="alert alert-block alert-info">
You may need to install the <tt>wget</tt> package. To install the <tt>wget</tt> package, run the following command.</div>

In [ ]:
!pip install --upgrade wget

In [10]:
# Download the sample PMML model and the 'iris_chaid.xml' file from the Github repo.
import wget
import os

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store the downloaded file as CHAID PMML model for Iris data, and then list all the saved models. First, you need to create the model metadata. The software specification required to save and deploy a PMML model is `spark-mllib_2.3`.

In [11]:
# Model metadata
software_spec_uid = client.software_specifications.get_uid_by_name('spark-mllib_2.3')
props_pmml = {
    client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: 'pmml_4.3'
}

<div class="alert alert-block alert-info">To list the supported software specifications, run <tt>client.software_specifications.list()</tt>.<br>To find more information about the frameworks with their respective <b>Types</b> and <b>Software Specifications</b>, visit the <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/wmls/wmls-deploy-python-types.html" target="_blank" rel="noopener no referrer">documentation</a>.</div>

You can extract the model UID from the saved model details.

In [12]:
# Create the model artifact.
model_artifact = client.repository.store_model(filename, meta_props=props_pmml)
model_uid = client.repository.get_model_uid(model_artifact)
print("Model UID = " + model_uid)

Model UID = 9043ae64-cc1a-416e-aafe-d7f093c502f9


Get the saved model metadata using the model UID.

In [13]:
# Details about the model.
model_details = client.repository.get_details(model_uid)
from pprint import pprint
pprint(model_details)

{'entity': {'content_status': {'state': 'persisted'},
            'name': 'CHAID PMML model for Iris data',
            'software_spec': {'id': '2e51f700-bca0-4b0d-88dc-5c6791338875'},
            'space': {'href': '/v4/spaces/d748238d-fd86-4db5-aa9d-3ebccad83f95',
                      'id': 'd748238d-fd86-4db5-aa9d-3ebccad83f95'},
            'type': 'pmml_4.3'},
 'metadata': {'created_at': '2020-03-16T21:16:58.002Z',
              'guid': '9043ae64-cc1a-416e-aafe-d7f093c502f9',
              'href': '/v4/models/9043ae64-cc1a-416e-aafe-d7f093c502f9?space_id=d748238d-fd86-4db5-aa9d-3ebccad83f95',
              'id': '9043ae64-cc1a-416e-aafe-d7f093c502f9',
              'modified_at': '2020-03-16T21:16:59.002Z',
              'name': 'CHAID PMML model for Iris data',
              'owner': '1000330999',
              'space_id': 'd748238d-fd86-4db5-aa9d-3ebccad83f95'}}


You can list all stored models once again using the `list_models` method.

In [14]:
# Display a list of all the models.
client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  --------
GUID                                  NAME                            CREATED                   TYPE
9043ae64-cc1a-416e-aafe-d7f093c502f9  CHAID PMML model for Iris data  2020-03-16T21:16:58.002Z  pmml_4.3
------------------------------------  ------------------------------  ------------------------  --------


<div class="alert alert-block alert-info">
From the list of stored models, you can see that model is successfully saved.<br><br>
If you've set the default project, this means you've saved the model in your project. You can see the saved model in your project UI by clicking on your project name in the breadcrumb at the top of the application.<br><br>
If you've set the default space, this means that you've saved the model in your deployment space. You can view your model by selecting <b>Analytics Deployments</b> under <b>Analyze</b> from the Navigation Menu and clicking on your deployment space name.</div>

If you're using a deployment space, proceed to Section 3: [Create an online deployment (with deployment space only)](#deploy). If not, you may skip to the [summary](#summary).

## 3. Create an online deployment (with deployment space only)<a id="deploy"></a>

Now, create an online deployment, *Iris species prediction*, for the stored model, and list all the online deployments for the model.

In [15]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Iris species prediction",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [16]:
# Create an online deployment.
deployment_details = client.deployments.create(model_uid, meta_props=deploy_meta)

# List the deployments.
client.deployments.list()



#######################################################################################

Synchronous deployment creation for uid: '9043ae64-cc1a-416e-aafe-d7f093c502f9' started

#######################################################################################


initializing.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='788abfcb-fa10-4761-98d5-d981c89a98d8'
------------------------------------------------------------------------------------------------


------------------------------------  -----------------------  -----  ------------------------  -------------
GUID                                  NAME                     STATE  CREATED                   ARTIFACT_TYPE
788abfcb-fa10-4761-98d5-d981c89a98d8  Iris species prediction  ready  2020-03-16T21:17:01.854Z  model
------------------------------------  -----------------------  -----  -----------------------

<div class="alert alert-block alert-info">
From the list of deployed models, you can see that model was successfully deployed in the deployment space.</div>

Now, you can check details of your deployments.

In [17]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment UID = {}'.format(deployment_uid))

Deployment uid = 788abfcb-fa10-4761-98d5-d981c89a98d8


In [18]:
# Deployment details.
print(client.deployments.get_details(deployment_uid))

{'metadata': {'parent': {'href': ''}, 'name': 'Iris species prediction', 'guid': '788abfcb-fa10-4761-98d5-d981c89a98d8', 'description': '', 'id': '788abfcb-fa10-4761-98d5-d981c89a98d8', 'modified_at': '2020-03-16T21:17:01.854Z', 'created_at': '2020-03-16T21:17:01.854Z', 'href': '/v4/deployments/788abfcb-fa10-4761-98d5-d981c89a98d8', 'space_id': 'd748238d-fd86-4db5-aa9d-3ebccad83f95'}, 'entity': {'name': 'Iris species prediction', 'custom': {}, 'online': {}, 'description': '', 'space': {'id': 'd748238d-fd86-4db5-aa9d-3ebccad83f95', 'href': '/v4/spaces/d748238d-fd86-4db5-aa9d-3ebccad83f95'}, 'status': {'state': 'ready', 'online_url': {'url': 'https://internal-nginx-svc:12443/v4/deployments/788abfcb-fa10-4761-98d5-d981c89a98d8/predictions'}}, 'asset': {'id': '9043ae64-cc1a-416e-aafe-d7f093c502f9', 'href': '/v4/models/9043ae64-cc1a-416e-aafe-d7f093c502f9?space_id=d748238d-fd86-4db5-aa9d-3ebccad83f95'}, 'space_id': 'd748238d-fd86-4db5-aa9d-3ebccad83f95'}}


### 3.1 Score a test data record<a id="score"></a>

Now, you can score the data and predict the species of iris flower given from the data.

In [19]:
# Score data and predict species of flower. Prepare scoring payload.
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'],
        'values': [[5.1, 3.5, 1.4, 0.2]]
    }]
}
pprint(job_payload)

{'input_data': [{'fields': ['Sepal.Length',
                            'Sepal.Width',
                            'Petal.Length',
                            'Petal.Width'],
                 'values': [[5.1, 3.5, 1.4, 0.2]]}]}


In [20]:
# Score data and predict species of flower. 
predictions = client.deployments.score(deployment_uid, job_payload)
print(predictions)

{'predictions': [{'fields': ['$R-Species', '$RC-Species', '$RP-Species', '$RP-setosa', '$RP-versicolor', '$RP-virginica', '$RI-Species'], 'values': [['setosa', 1.0, 1.0, 1.0, 0.0, 0.0, '1']]}]}


As we can see from the prediction, this is a species of Iris Setosa.

<a id="summary"></a>
## 4. Summary and next steps

You successfully completed this notebook! You learned how to use Watson Machine Learning for PMML model deployment and scoring. 

### Resources <a id="resources"></a>

To learn more about configurations used in this notebook or more sample notebooks, tutorials, documentation, how-tos, and blog posts, check out these links:

<div class="alert alert-block alert-success">

<h4>IBM documentation</h4>
<br>
 <li> <a href="https://wml-api-pyclient-dev-v4.mybluemix.net" target="_blank" rel="noopener no referrer">watson-machine-learning</a></li> 
 <li> <a href="https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/cpd/overview/welcome.html" target="_blank" rel="noopener noreferrer">CP4D 3.0</a></li>
 
<h4> IBM Samples</h4>
<br>
 <li> <a href="https://github.com/IBMDataScience/sample-notebooks" target="_blank" rel="noopener noreferrer">Sample notebooks</a></li>
 <li> <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener noreferrer">Sample models</a></li>
 
<h4> Others</h4>
<br>
 <li> <a href="https://www.python.org" target="_blank" rel="noopener noreferrer">Official Python website</a><br></li>
</div>

### Citation

Dua, D. and Karra Taniskidou, E. (2017). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml). Irvine, CA: University of California, School of Information and Computer Science.

### Authors

**Wojciech Sobala** is a Data Scientist at IBM.  <br><br>
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2018-2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>